In [1]:
import tabula
import pandas as pd

In [2]:
pdf_path = ""
first_page_coordinates = [350,35,729,559]
page_coordinates = [79,35,722,559]

In [3]:
total_pages = len(tabula.read_pdf(pdf_path,pages="all",stream=True,guess=False))

def get_data(page_number,coordinates):
    area_option = f"--area {','.join(map(str, coordinates))}"
    page_option = f"--pages {page_number}"
    return tabula.read_pdf(pdf_path,pages="all",options=f"{area_option} {page_option}",lattice=True,guess=False)

In [13]:
mpesa_page_one = get_data(1,first_page_coordinates)[0]
mpesa_remaining = []

for page in range(2,total_pages-1):
    mpesa_rest = get_data(page,page_coordinates)[0]
    mpesa_remaining.append(mpesa_rest)

last_page = tabula.read_pdf(pdf_path,pages = total_pages,lattice=True,guess=False)

In [ ]:
mpesa_combined = pd.concat([mpesa_page_one]+mpesa_remaining+last_page)
mpesa_combined